In [1]:
%pip install wikipedia anthropic

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 187 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 128 kB 46.4 MB/s eta 0:00:01
     |████████████████████████████████| 201 kB 60.7 MB/s eta 0:00:01
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=e69367a83668dcdb973efc7126c4d69e84a0c12df03f88c3af99a280a5d38879
  Stored in directory: /Users/anshika@browserstack.com/Library/Caches/pip/wheels/c2/46/f4/caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import wikipedia
import os

def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    os.makedirs("output", exist_ok=True)
    with open("output/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")


/Users/anshika@browserstack.com/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
tool_definitions = [
    {
        "name": "generate_wikipedia_reading_list",
        "description": "Takes a research topic and a list of potential article titles, and finds real Wikipedia articles related to the topic. Then saves them to a markdown file for later reading.",
        "input_schema": {
            "type": "object",
            "properties": {
                "research_topic": {
                    "type": "string",
                    "description": "The topic the user is researching"
                },
                "article_titles": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A list of possible Wikipedia article titles"
                }
            },
            "required": ["research_topic", "article_titles"]
        }
    }
]


In [ ]:
from anthropic import Anthropic
import os
import json

client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

def get_research_help(topic, num_articles=3):
    system_prompt = (
        "You are a helpful research assistant. When a user gives you a research topic, "
        "suggest a list of realistic Wikipedia article titles related to the topic. "
        f"Please return exactly {num_articles} titles in a JSON array, like: [\"Title 1\", \"Title 2\"]"
    )

    user_prompt = f"I need help researching the topic '{topic}'. Please give me {num_articles} Wikipedia article titles."

    response = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1024,
        temperature=0.7,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt}]
    )

    message = response.content[0].text.strip()

    try:
        article_titles = json.loads(message)
        if isinstance(article_titles, list):
            generate_wikipedia_reading_list(topic, article_titles)
            print(f"Added articles for topic: {topic}")
        else:
            raise ValueError("Response was not a list")
    except Exception as e:
        print("Failed to parse Claude's output as JSON. Here's the raw output:")
        print(message)
        print("Error:", e)


In [12]:
get_research_help("History of Hawaii", 3)
get_research_help("Pirates Across The World", 5)
get_research_help("Are animals conscious?", 4)

Added articles for topic: History of Hawaii
Failed to parse Claude's output as JSON. Here's the raw output:
Here are 5 relevant Wikipedia article titles for the topic 'Pirates Across The World':

["History of Piracy", "Golden Age of Piracy", "List of Pirate Flags", "Piracy in the Caribbean", "Piracy in Southeast Asia"]
Error: Expecting value: line 1 column 1 (char 0)
Failed to parse Claude's output as JSON. Here's the raw output:
Here are 4 Wikipedia article titles related to the topic "Are animals conscious?":

["Consciousness in animals", "Animal cognition", "Sentience", "Philosophy of animal mind"]
Error: Expecting value: line 1 column 1 (char 0)
